In [3]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timezone


# so a plan could be that I already have a bunch of humidity data, how about I just test with that?
# the distribution sould be somewhat similar to some other sensors
test_data_folder = "/home/chowder/Documents/unifiedSensorClient/writers/tests/test_humidity_data/"

#get all the files in the test_data_folder
files = os.listdir(test_data_folder)

#load all the files and concatenate them
humidity_data = pd.concat([pd.read_parquet(os.path.join(test_data_folder, f)) for f in files])


/tmp/ipykernel_3633285/2928142081.py:16: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  timestamps = pd.date_range(start=start_time, periods=num_samples, freq=f'{1/data_hz}S')


In [4]:
humidity_data

,relativeHumidity!float32!float32!afloat5
sampleDT!int64!datetime64[ns]!audelayhz0,
2025-05-01 13:46:37+00:00,37.50000
2025-05-01 13:46:38+00:00,38.03125
2025-05-01 13:46:39+00:00,38.09375
2025-05-01 13:46:40+00:00,38.09375
2025-05-01 13:46:41+00:00,37.53125
...,...
2025-05-01 13:46:33+00:00,37.75000
2025-05-01 13:46:34+00:00,37.78125
2025-05-01 13:46:35+00:00,37.53125


In [10]:
# so we'll be generating a parquet file with timestamps and some data

simulated_data_hz = 32
#start at midnight on january 7th 2026
simulated_start_time = datetime(2026, 1, 7, 0, 0, 0, 0, timezone.utc)

#we will generate 10 hours of data
num_simulated_samples =24 * 60 * 60 * simulated_data_hz

#create a dataframe with the timestamps
simulated_timestamps = pd.date_range(start=simulated_start_time, periods=num_simulated_samples, freq=f'{1/simulated_data_hz}s')

#create a dataframe with the timestamps and some data
# Ensure that 'data' is a 1D array or Series of appropriate values
simulated_data = pd.DataFrame({
    'timestamp': simulated_timestamps,
    'data': humidity_data.iloc[:num_simulated_samples].values.squeeze()  # ensure 1D shape
})

simulated_data


,timestamp,data
0,2026-01-07 00:00:00+00:00,37.50000
1,2026-01-07 00:00:00.031250+00:00,38.03125
2,2026-01-07 00:00:00.062500+00:00,38.09375
3,2026-01-07 00:00:00.093750+00:00,38.09375
4,2026-01-07 00:00:00.125000+00:00,37.53125
...,...,...
2764795,2026-01-07 23:59:59.843750+00:00,36.78125
2764796,2026-01-07 23:59:59.875000+00:00,36.68750
2764797,2026-01-07 23:59:59.906250+00:00,36.71875
2764798,2026-01-07 23:59:59.937500+00:00,36.96875


In [13]:
def generate_wavpak_test_data(data_loc, data_hz, start_time, duration_seconds):
    files = os.listdir(data_loc)
    data = pd.concat([pd.read_parquet(os.path.join(data_loc, f)) for f in files])
    num_samples = duration_seconds * data_hz
    timestamps = pd.date_range(start=start_time, periods=num_samples, freq=f'{1/data_hz}s')
    data = data.iloc[:num_samples].reset_index(drop=True).values.squeeze()
    return pd.DataFrame({'timestamp': timestamps, 'data': data})

generate_wavpak_test_data(test_data_folder, 32, datetime(2026, 1, 7, 0, 0, 0, 0, timezone.utc), 24 * 60 * 60)


,timestamp,data
0,2026-01-07 00:00:00+00:00,37.50000
1,2026-01-07 00:00:00.031250+00:00,38.03125
2,2026-01-07 00:00:00.062500+00:00,38.09375
3,2026-01-07 00:00:00.093750+00:00,38.09375
4,2026-01-07 00:00:00.125000+00:00,37.53125
...,...,...
2764795,2026-01-07 23:59:59.843750+00:00,36.78125
2764796,2026-01-07 23:59:59.875000+00:00,36.68750
2764797,2026-01-07 23:59:59.906250+00:00,36.71875
2764798,2026-01-07 23:59:59.937500+00:00,36.96875
